In [ ]:
!pip install mtcnn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 23.4 MB/s eta 0:00:00


In [ ]:
import os
import cv2
from mtcnn import MTCNN
import torch
from torchvision import transforms
from PIL import Image
import json

In [ ]:
# Initialize MTCNN
detector = MTCNN()

In [ ]:
# Preprocessing function
def preprocess_image(image_path, detector):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    result = detector.detect_faces(image)
    if result:
        keypoints = result[0]['keypoints']
        bounding_box = result[0]['box']
        x, y, width, height = bounding_box
        face = image[y:y+height, x:x+width]
        face_resized = cv2.resize(face, (112, 112))
        face_tensor = transforms.ToTensor()(face_resized)

        return face_tensor, keypoints
    else:
        return None, None

In [ ]:
dataset_path = '/content/drive/MyDrive/DL/data/raw'
new_dataset_path = '/content/drive/MyDrive/DL/data/new_dataset'
keypoints_path = '/content/drive/MyDrive/DL/data/keypoints'

In [ ]:
for subdir, dirs, files in os.walk(dataset_path):
    for file in files:
        input_image_path = os.path.join(subdir, file)
        try:
            preprocessed_face, keypoints = preprocess_image(input_image_path, detector)
            if preprocessed_face is not None:
                save_subdir = os.path.relpath(subdir, dataset_path)
                save_dir = os.path.join(new_dataset_path, save_subdir)
                os.makedirs(save_dir, exist_ok=True)
                save_image_path = os.path.join(save_dir, file)
                pil_image = transforms.ToPILImage()(preprocessed_face).convert("RGB")
                pil_image.save(save_image_path)

                keypoints_dir = os.path.join(keypoints_path, save_subdir)
                os.makedirs(keypoints_dir, exist_ok=True)
                keypoints_file = os.path.splitext(file)[0] + '.json'
                keypoints_file_path = os.path.join(keypoints_dir, keypoints_file)
                with open(keypoints_file_path, 'w') as f:
                    json.dump(keypoints, f)

        except Exception as e:
            print(f"Error processing {input_image_path}: {e}")

print("Preprocessing and keypoints extraction complete.")


1/1 [==============================] - 0s 30ms/step
Preprocessing and keypoints extraction complete.
